In [1]:
import os
import sys
from pathlib import Path
import logging
import time
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client
import hotspot_utils as util
import process_nearest_hotspots as nearest_process
import logging

In [2]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
_LOG = logging.getLogger(__name__)

In [3]:
client = Client(processes=False)

# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The parameter `compare_field` is the name of column that will be used to generate nearest hotspots, the data in the column should be of datetime type.

In [4]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "dea_frp": None,
    "lon_west": 147.0,
    "lat_south": -38.0,
    "lon_east": 154.0,
    "lat_north": -27.,
    "start_date": "2019-11-01",
    "end_date": "2020-10-08",
    "start_time": "21:00",
    "end_time": "3:00",
    "chunks": 300,
    "outdir": Path(os.getcwd()).joinpath("workdir"),
    "compare_field": "solar_night"
}

In [ ]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

2021-03-12 02:25:49 [INFO] process_nearest_hotspots - Processing Neareast Hotspots...
2021-03-12 02:25:49 [INFO] botocore.credentials - Found credentials in environment variables.
2021-03-12 02:25:49 [INFO] hotspot_utils - s3://s3vtaustralia/nasa_hotspots_gdf.geojson exists: skipped download
2021-03-12 02:25:49 [INFO] hotspot_utils - s3://s3vtaustralia/s3vt_hotspots.geojson exists: skipped download
2021-03-12 02:25:49 [INFO] hotspot_utils - s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson exists: skipped download
2021-03-12 02:25:49 [INFO] hotspot_utils - s3://s3vtaustralia/landgate_hotspots_gdf.geojson exists: skipped download
2021-03-12 02:25:49 [INFO] hotspot_utils - dea Hotspots FRP  is None. excluding from analysis.
2021-03-12 02:25:49 [INFO] process_nearest_hotspots - Reading spatial and temporal subsets of all hotspots dataframes...
2021-03-12 02:25:49 [INFO] hotspot_utils - reading and subsetting GeoDataFrame for nasa: /home/jovyan/s3vt_dask/s3vtdata/workdir/nasa_hotspots_gdf.

/env/lib/python3.6/site-packages/distributed/worker.py:3382: UserWarning: Large object of size 1.99 MB detected in task graph: 
  (                     latitude  longitude satellit ... 7/2019-12-04'))
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


2021-03-12 02:40:04 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and SUOMI NPP_VIIRS_NASA1
2021-03-12 02:44:07 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and TERRA_MODIS_NASA6.03
2021-03-12 02:45:27 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and AQUA_MODIS_NASA6.03
2021-03-12 02:46:39 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and SENTINEL_3A_SLSTR_ESA
2021-03-12 02:47:49 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and SENTINEL_3B_SLSTR_ESA
2021-03-12 02:49:05 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and SENTINEL_3A_SLSTR_EUMETSAT
2021-03-12 02:50:46 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.0NRT and SENTINEL_3B_SLSTR_EUMETSAT
2021-03-12 02:52:37 [INFO] process_nearest_hotspots - Comparing Hotspots for NOAA 20_VIIRS_NASA2.

## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [ ]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = Path(os.getcwd()).joinpath("workdir")
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith(f"{processing_parameters['compare_field']}.csv"))
]

In [ ]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

In [ ]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [ ]:
nearest_points_ddf.head()

# Results
## Co-occurrence metrics

In [ ]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [ ]:
nearest_ddf_dist_subset = client.persist(nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold])

In [ ]:
# Count of hotspot matches < dist_threshold
numerator = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="count",
    aggfunc="count"
).compute()

In [ ]:
numerator.astype(int)

In [ ]:
# Count of hotspot matches - total  
denominator = util.dask_pivot_table(
        nearest_points_ddf,
        index="2_satellite_sensor_product",
        column="satellite_sensor_product",
        values="count",
        aggfunc="count",
    ).compute()

In [ ]:
denominator.astype(int)

In [ ]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [ ]:
difference.astype(int)

In [ ]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100

In [ ]:
percentage.astype(int)

In [ ]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [ ]:
timemax

In [ ]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
    
)

In [ ]:
timemin

In [ ]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="dist_m",
    aggfunc="mean",
).compute()

In [1]:
np.round(averagedist, 2)

NameError: name 'np' is not defined